In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time
from kaggle.api.kaggle_api_extended import KaggleApi
import pickle

c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_DQ.csv'


ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202106

# agregue sus semillas
semillas = [540079, 250829, 314299, 302111, 801007]

data = pd.read_csv(dataset_path + dataset_file)

C:\Users\Federico\AppData\Local\Temp\ipykernel_19900\3249954165.py:17: DtypeWarning: Columns (97) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dataset_path + dataset_file)


In [3]:
df=data.copy()
df.shape

(981946, 98)

In [4]:
df['clase_peso'] = 1.0
df['clase_binaria'] = 0
df['clase_binaria'] = np.where(df['clase_ternaria'] == 'CONTINUA', 0, 1)

df.loc[df['clase_binaria'] == 1, 'clase_peso'] = 1.0001

In [5]:
df[df["foto_mes"]==202106]

,mpayroll,cpayroll_trx,mcuentas_saldo,mrentabilidad_annual,mactivos_margen,mtarjeta_visa_consumo,ctrx_quarter,mpasivos_margen,mrentabilidad,Visa_mpagominimo,...,cprestamos_prendarios,ctarjeta_visa,Master_msaldodolares,mttarjeta_master_debitos_automaticos,Master_mconsumototal,tmobile_app,foto_mes,clase_ternaria,clase_peso,clase_binaria
817070,117300.00,1,20016.94,20332.88,-2491.60,20708.29,199,1029.14,399.88,1313.76,...,0,1,0.00,9328.57,53339.61,0,202106,NaN,1.0001,1
817071,0.00,0,130909.79,23227.25,-2810.24,113842.54,191,2103.70,4708.37,0.00,...,0,1,450.42,0.00,36861.75,0,202106,NaN,1.0001,1
817072,235650.91,2,111111.85,38207.62,1180.59,14491.15,172,1376.77,3534.50,18885.30,...,0,1,NaN,0.00,NaN,0,202106,NaN,1.0001,1
817073,0.00,0,5.05,1445.30,-678.84,21741.23,20,204.32,-138.82,1818.15,...,0,1,0.00,0.00,NaN,0,202106,NaN,1.0001,1
817074,53958.00,2,58408.06,36468.62,-659.77,50919.03,29,721.51,556.99,4058.58,...,0,1,0.00,0.00,NaN,0,202106,NaN,1.0001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,23460.00,1,0.00,0.00,0.00,0.00,2,0.00,0.00,0.00,...,0,1,0.00,0.00,NaN,0,202106,NaN,1.0001,1
981942,0.00,1,91576.57,95.20,0.00,0.00,5,81.16,95.20,0.00,...,0,1,0.00,0.00,NaN,0,202106,NaN,1.0001,1
981943,0.00,1,71850.94,72.08,0.00,0.00,2,61.45,72.08,0.00,...,0,1,0.00,0.00,NaN,0,202106,NaN,1.0001,1
981944,116570.62,2,0.00,0.00,0.00,0.00,2,0.00,0.00,0.00,...,0,1,0.00,0.00,NaN,0,202106,NaN,1.0001,1


In [6]:
## funciones generales
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.0001, ganancia_acierto, 0) - np.where(weight < 1.0001, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [7]:
train_data = df[df['foto_mes'] == mes_train]
test_data = df[df['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = train_data['clase_binaria']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)


In [8]:
X_train.shape

(164090, 97)

In [ ]:
#optimizacion

In [9]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 20, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.05, 0.3) # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 2000, 6000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.3, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 1.0)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': 1,
        'early_stopping_rounds': int(50 + 5 / learning_rate),
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # todos los baja como 1
                              weight=w_train)
    
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=100000, # modificar, subit y subir
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


storage_name = "sqlite:///" + db_path + "optimizacion_LGBM_competencia_01.db"
study_name = "modelo_11"

study_1= optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-10-08 22:40:41,734] A new study created in RDB with name: modelo_11


In [10]:
study_1.optimize(objective, n_trials=100) # subir subir

[I 2024-10-08 22:40:54,407] Trial 0 finished with value: 350140000.0 and parameters: {'num_leaves': 318, 'learning_rate': 0.24310628352475655, 'min_data_in_leaf': 2745, 'feature_fraction': 0.5444495248444603, 'bagging_fraction': 0.8772486156456734}. Best is trial 0 with value: 350140000.0.
[I 2024-10-08 22:41:08,124] Trial 1 finished with value: 357574000.0 and parameters: {'num_leaves': 377, 'learning_rate': 0.06946764611792959, 'min_data_in_leaf': 3669, 'feature_fraction': 0.73674744595699, 'bagging_fraction': 0.49924492800984355}. Best is trial 1 with value: 357574000.0.
[W 2024-10-08 22:41:16,219] Trial 2 failed with parameters: {'num_leaves': 312, 'learning_rate': 0.18939404763521, 'min_data_in_leaf': 5850, 'feature_fraction': 0.707447433926303, 'bagging_fraction': 0.7686270396066526} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py",

KeyboardInterrupt: 

In [11]:
optuna.visualization.plot_optimization_history(study_1)

In [12]:
plot_param_importances(study_1)

In [13]:
plot_slice(study_1)

In [9]:
storage_name = "sqlite:///" + db_path + "optimizacion_LGBM_competencia_01.db"
study_name = "modelo_11"

study_1= optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-10-08 23:38:40,161] Using an existing study with name 'modelo_11' instead of creating a new one.


In [10]:
def entrenar_modelos_con_varias_semillas(best_params, X_train, y_train_binaria, w_train, best_iter, semillas):
    """
    Entrena modelos de LightGBM utilizando los mejores parámetros y una lista de semillas.
    
    Parámetros:
    - best_params: dict, contiene los mejores parámetros del estudio de optimización.
    - X_train: dataframe, las características de entrenamiento.
    - y_train_binaria: array-like, la variable objetivo binaria.
    - w_train: array-like, los pesos para las observaciones de entrenamiento.
    - best_iter: int, la mejor cantidad de árboles según el estudio de optimización.
    - semillas: lista de int, las semillas para entrenar cada modelo.
    
    Retorna:
    - modelos_entrenados: diccionario de modelos de LightGBM entrenados, con nombres basados en la semilla.
    """
    modelos_entrenados = {}

    # Crear el dataset de entrenamiento
    train_data = lgb.Dataset(X_train, label=y_train_binaria, weight=w_train)

    # Entrenar un modelo por cada semilla
    for seed in semillas:
        # Actualizar el parámetro de la semilla
        params = best_params.copy()
        params['seed'] = seed
        
        # Entrenar el modelo
        model = lgb.train(params, train_data, num_boost_round=best_iter)
        
        # Asignar el nombre del modelo como "modelo_semilla"
        nombre_modelo = f"modelo_{seed}"
        modelos_entrenados[nombre_modelo] = model
        
        print(f"Modelo entrenado con semilla {seed} guardado como {nombre_modelo}")
    
    return modelos_entrenados

In [11]:
study=study_1
best_iter=study_1.best_trial.user_attrs["best_iter"]

# Definir los mejores parámetros del estudio
best_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'verbose': 0
}


# Entrenar los modelos
modelos_entrenado = entrenar_modelos_con_varias_semillas(best_params, X_train, y_train_binaria, w_train, best_iter, semillas)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [18]:
modelos_entrenado

{'modelo_540079': <lightgbm.basic.Booster at 0x2b731a058b0>,
 'modelo_250829': <lightgbm.basic.Booster at 0x2b731ac5850>,
 'modelo_314299': <lightgbm.basic.Booster at 0x2b731a44f50>,
 'modelo_302111': <lightgbm.basic.Booster at 0x2b7319ad370>,
 'modelo_801007': <lightgbm.basic.Booster at 0x2b7300a5dc0>}

In [22]:
def predecir_y_ordenar_modelos(modelos_entrenados, X_test):
    """
    Realiza predicciones con modelos entrenados de LightGBM, agrega la probabilidad de ser "BAJA" 
    y ordena los clientes de forma descendente por probabilidad.
    
    Parámetros:
    - modelos_entrenados: dict, contiene los modelos entrenados con las semillas como nombres.
    - X_test: dataframe, conjunto de test.
    
    Retorna:
    - resultados_predicciones: diccionario con los resultados predichos y ordenados para cada modelo entrenado.
    """
    resultados_predicciones = {}

    # Asegurarse de que X_test tenga las mismas columnas que los datos de entrenamiento
    columnas_modelo = modelos_entrenados[list(modelos_entrenados.keys())[0]].feature_name()
    X_test = X_test[columnas_modelo]

    # Asegurarse de que 'numero_de_cliente' esté en X_test
    if 'numero_de_cliente' not in X_test.columns:
        raise ValueError("La columna 'numero_de_cliente' debe estar presente en X_test")

    # Para cada modelo entrenado, hacemos predicciones y ordenamos los resultados
    for nombre_modelo, model in modelos_entrenados.items():
        try:
            # Hacer predicciones sobre el conjunto de test
            predicciones = model.predict(X_test)
            
            # Crear un nuevo DataFrame solo con 'numero_de_cliente' y 'Probabilidad'
            resultados = pd.DataFrame({
                'numero_de_cliente': X_test['numero_de_cliente'],
                'Probabilidad': predicciones
            })
            
            # Ordenar a los clientes por la probabilidad de ser "BAJA"
            resultados_ordenados = resultados.sort_values(by='Probabilidad', ascending=False)
            
            # Guardar los resultados ordenados
            resultados_predicciones[nombre_modelo] = resultados_ordenados
            
            print(f"Predicciones realizadas y ordenadas para {nombre_modelo}")
        
        except Exception as e:
            print(f"Error al procesar {nombre_modelo}: {str(e)}")
            continue
    
    return resultados_predicciones

In [33]:
def predecir_y_ordenar_modelos2(modelos_entrenados, X_test):
    """
    Realiza predicciones con modelos entrenados de LightGBM y crea un DataFrame final
    con la probabilidad más alta obtenida por cada cliente en cualquiera de los modelos.
    
    Parámetros:
    - modelos_entrenados: dict, contiene los modelos entrenados con las semillas como nombres.
    - X_test: dataframe, conjunto de test.
    
    Retorna:
    - df_final: DataFrame con 'numero_de_cliente' y la 'Probabilidad' más alta para cada cliente.
    """
    # Asegurarse de que X_test tenga las mismas columnas que los datos de entrenamiento
    columnas_modelo = modelos_entrenados[list(modelos_entrenados.keys())[0]].feature_name()
    X_test = X_test[columnas_modelo]

    # Asegurarse de que 'numero_de_cliente' esté en X_test
    if 'numero_de_cliente' not in X_test.columns:
        raise ValueError("La columna 'numero_de_cliente' debe estar presente en X_test")

    # Crear un DataFrame para almacenar todas las predicciones
    todas_predicciones = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente']})

    # Para cada modelo entrenado, hacemos predicciones
    for nombre_modelo, model in modelos_entrenados.items():
        try:
            # Hacer predicciones sobre el conjunto de test
            predicciones = model.predict(X_test)
            
            # Agregar las predicciones al DataFrame
            todas_predicciones[nombre_modelo] = predicciones
            
            print(f"Predicciones realizadas para {nombre_modelo}")
        
        except Exception as e:
            print(f"Error al procesar {nombre_modelo}: {str(e)}")
            continue
    
    # Calcular la probabilidad máxima para cada cliente
    todas_predicciones['Probabilidad'] = todas_predicciones.iloc[:, 1:].max(axis=1)
    
    # Crear el DataFrame final con 'numero_de_cliente' y la 'Probabilidad' más alta
    df_final = todas_predicciones[['numero_de_cliente', 'Probabilidad']]
    
    # Ordenar el DataFrame final por 'Probabilidad' de forma descendente
    df_final = df_final.sort_values(by='Probabilidad', ascending=False)

    return df_final

In [37]:
predicciones_1=predecir_y_ordenar_modelos(modelos_entrenado, X_test)

Predicciones realizadas y ordenadas para modelo_540079
Predicciones realizadas y ordenadas para modelo_250829
Predicciones realizadas y ordenadas para modelo_314299
Predicciones realizadas y ordenadas para modelo_302111
Predicciones realizadas y ordenadas para modelo_801007


In [38]:
df_final=predecir_y_ordenar_modelos2(modelos_entrenado, X_test)

Predicciones realizadas para modelo_540079
Predicciones realizadas para modelo_250829
Predicciones realizadas para modelo_314299
Predicciones realizadas para modelo_302111
Predicciones realizadas para modelo_801007


In [39]:
df_modelo_540079 = pd.DataFrame(predicciones_1['modelo_540079'])
df_modelo_250829 = pd.DataFrame(predicciones_1['modelo_250829'])
df_modelo_314299 = pd.DataFrame(predicciones_1['modelo_314299'])
df_modelo_302111 = pd.DataFrame(predicciones_1['modelo_302111'])
df_modelo_801007 = pd.DataFrame(predicciones_1['modelo_801007'])

In [51]:
df_modelo_302111

,numero_de_cliente,Probabilidad
910079,812656770,0.890022
978032,1462097870,0.878702
856107,511863915,0.860759
977205,1455714205,0.847359
828791,303620250,0.845073
...,...,...
861297,554981460,0.000040
917847,857932152,0.000040
845849,464825086,0.000039
901635,752639620,0.000031


In [48]:
df_final

,numero_de_cliente,Probabilidad
910079,812656770,0.890022
978032,1462097870,0.878702
873626,606371992,0.866076
856107,511863915,0.865121
977205,1455714205,0.847359
...,...,...
823738,280167996,0.000061
888399,674618747,0.000059
883807,652025650,0.000059
869419,591228473,0.000057


In [52]:
api = KaggleApi()
api.authenticate()

In [53]:
# #L. Predecimos Junio.
# #i. Predecimos propiamente dicho.
# predicciones = model_lgb.predict(X_test_m1)
# #ii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
# X_test_m1['Probabilidad'] = predicciones
# #iii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
# tb_entrega = X_test_m1.sort_values(by='Probabilidad', ascending=False)
#iv. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
tb_entrega=df_final
cortes = range(11000,12000,100)
#v. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
num_subida_kaggle = 1
for envios in cortes:
    #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    #2. Guardamos el archivo para Kaggle.
    nombre_archivo = "Primer_ensable_menos_rango{}.csv".format(num_subida_kaggle)
    ruta_archivo= "../../../exp/{}".format(nombre_archivo)
    resultados.to_csv(ruta_archivo, index=False)
    
    num_subida_kaggle += 1
    
    #3. Envío a Kaggle.
    #a. Defino los parámetros claves.
    mensaje = f'Archivo {nombre_archivo}.Punto_corte: {envios}.'
    competencia = 'dm-ey-f-2024-primera'
    #c. Subo la Submission.
    while True:
        try:
            api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
            print("Submission successful!")
            break
        except ApiException as e:
            print(f"Error: {e}")  # Imprime la excepción completa para ver qué atributos tiene
            if e.status == 429:  # Reemplaza esto si `status` no es correcto
                print("Rate limit exceeded. Retrying after 30 seconds...")
                time.sleep(30)
            else:
                raise e  # Re-raise other exceptions

Cantidad de clientes 11000


100%|██████████| 2.08M/2.08M [00:01<00:00, 1.32MB/s]


Submission successful!
Cantidad de clientes 11100


100%|██████████| 2.08M/2.08M [00:01<00:00, 1.34MB/s]


Submission successful!
Cantidad de clientes 11200


...

Submission successful!
Cantidad de clientes 11800


100%|██████████| 2.08M/2.08M [00:01<00:00, 1.38MB/s]


Submission successful!
Cantidad de clientes 11900


100%|██████████| 2.08M/2.08M [00:01<00:00, 1.31MB/s]


Submission successful!
